In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import FloatSlider, IntSlider, Dropdown, Button, VBox, HBox
import warnings

warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.quantum_info import Statevector, partial_trace, entropy
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
class QuantumAISimulator:
    def __init__(self, n_qubits=3):
        self.n_qubits = n_qubits
        self.pennylane_device = qml.device('default.qubit', wires=n_qubits)
        self.current_state = None
        print(f"Simulator initialized with {n_qubits} qubits")

    def create_circuit(self, gates_config):
        qc = QuantumCircuit(self.n_qubits)
        for gate in gates_config:
            gate_type = gate['type']
            qubit = gate['qubit']
            if gate_type == 'H': qc.h(qubit)
            elif gate_type == 'X': qc.x(qubit)
            elif gate_type == 'Y': qc.y(qubit)
            elif gate_type == 'Z': qc.z(qubit)
            elif gate_type == 'RX' and 'angle' in gate: qc.rx(gate['angle'], qubit)
            elif gate_type == 'RY' and 'angle' in gate: qc.ry(gate['angle'], qubit)
            elif gate_type == 'RZ' and 'angle' in gate: qc.rz(gate['angle'], qubit)
            elif gate_type == 'CNOT' and 'target' in gate and qubit != gate['target']: qc.cx(qubit, gate['target'])
        return qc

    def execute_circuit(self, circuit, shots=1024):
        statevector = Statevector.from_instruction(circuit)
        probs = np.abs(statevector.data) ** 2
        n = self.n_qubits
        all_states = [f"{i:0{n}b}" for i in range(2**n)]
        samples = np.random.choice(all_states, size=shots, p=probs)
        counts = {}
        for s in samples:
            counts[s] = counts.get(s, 0) + 1
        self.current_state = statevector
        return {
            'statevector': statevector,
            'counts': counts,
            'circuit': circuit
        }

    def calculate_entanglement_entropy(self, statevector):
        try:
            # For 3 qubits trace out qubits 1 and 2 to get reduced density matrix of qubit 0
            traced_out_qubits = [1, 2] if self.n_qubits == 3 else [i for i in range(1, self.n_qubits)]
            reduced_rho = partial_trace(statevector, traced_out_qubits)
            entropy_val = entropy(reduced_rho)
            return min(entropy_val, 1.0)  # Normalize to max 1
        except Exception as e:
            # In case of error, return 0 entropy
            return 0.0


    def visualize_quantum_state(self, result, show_bloch=True, show_histogram=True):
        fig = plt.figure(figsize=(15, 10))

        # Circuit Diagram
        ax1 = plt.subplot(3, 3, (1, 3))
        try:
            circuit_img = circuit_drawer(result['circuit'], output='mpl', style={'backgroundcolor': 'white'})
            ax1.imshow(circuit_img)
            ax1.axis('off')
            ax1.set_title('Quantum Circuit', fontsize=14, fontweight='bold')
        except Exception:
            # Fallback to text if mpl drawing is unavailable
            ax1.text(0.05, 0.95, str(result['circuit'].draw(output='text')),
                     fontsize=8, fontfamily='monospace', va='top', ha='left', wrap=True, transform=ax1.transAxes)
            ax1.axis('off')
            ax1.set_title('Quantum Circuit (Text)', fontsize=14, fontweight='bold')

        # State Vector Probabilities
        ax2 = plt.subplot(3, 3, 4)
        statevector = result['statevector']
        amplitudes = np.abs(statevector.data)**2
        states = [f'|{i:0{self.n_qubits}b}⟩' for i in range(len(amplitudes))]
        bars = ax2.bar(range(len(amplitudes)), amplitudes,
                       color=plt.cm.viridis(amplitudes / max(amplitudes) if max(amplitudes) > 0 else 1))
        ax2.set_xlabel('Quantum States')
        ax2.set_ylabel('Probability')
        ax2.set_title('State Vector Probabilities')
        ax2.set_xticks(range(len(states)))
        ax2.set_xticklabels(states, rotation=45)
        for bar, amp in zip(bars, amplitudes):
            if amp > 0.01:
                ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                         f'{amp:.3f}', ha='center', va='bottom', fontsize=8)

        # Measurement Histogram
        if show_histogram:
            ax3 = plt.subplot(3, 3, 5)
            counts = result['counts']
            all_states = [f'{i:0{self.n_qubits}b}' for i in range(2 ** self.n_qubits)]
            count_values = [counts.get(state, 0) for state in all_states]
            bars = ax3.bar(range(len(all_states)), count_values,
                           color=plt.cm.plasma(
                               np.array(count_values) / (max(count_values) if max(count_values) > 0 else 1)))
            ax3.set_xlabel('Measured States')
            ax3.set_ylabel('Counts')
            ax3.set_title('Measurement Results')
            ax3.set_xticks(range(len(all_states)))
            ax3.set_xticklabels([f'|{state}⟩' for state in all_states], rotation=45)

        # Bloch Sphere (Qubit 0)
        if show_bloch and self.n_qubits >= 1:
            ax4 = plt.subplot(3, 3, 6, projection='3d')
            try:
                rho = partial_trace(statevector, [i for i in range(1, self.n_qubits)])
                pauli_x = np.array([[0, 1], [1, 0]])
                pauli_y = np.array([[0, -1j], [1j, 0]])
                pauli_z = np.array([[1, 0], [0, -1]])
                x = np.real(np.trace(rho.data @ pauli_x))
                y = np.real(np.trace(rho.data @ pauli_y))
                z = np.real(np.trace(rho.data @ pauli_z))
                u = np.linspace(0, 2 * np.pi, 50)
                v = np.linspace(0, np.pi, 50)
                sphere_x = np.outer(np.cos(u), np.sin(v))
                sphere_y = np.outer(np.sin(u), np.sin(v))
                sphere_z = np.outer(np.ones(np.size(u)), np.cos(v))
                ax4.plot_surface(sphere_x, sphere_y, sphere_z, alpha=0.3, color='lightblue')
                ax4.quiver(0, 0, 0, x, y, z, color='red', linewidth=3, arrow_length_ratio=0.1)
                ax4.quiver(0, 0, 0, 1.2, 0, 0, color='black', alpha=0.6)
                ax4.quiver(0, 0, 0, 0, 1.2, 0, color='black', alpha=0.6)
                ax4.quiver(0, 0, 0, 0, 0, 1.2, color='black', alpha=0.6)
                ax4.set_xlim([-1.2, 1.2])
                ax4.set_ylim([-1.2, 1.2])
                ax4.set_zlim([-1.2, 1.2])
                ax4.set_title('Bloch Sphere (Qubit 0)')
            except Exception:
                ax4.text(0.5, 0.5, 0.5, 'Bloch Sphere Unavailable', ha='center', va='center')

        # Quantum Metrics
        ax5 = plt.subplot(3, 3, (7, 9))
        ax5.axis('off')
        entanglement = self.calculate_entanglement_entropy(statevector)
        fidelity = np.abs(statevector.data[0]) ** 2
        coherence = np.sum(np.abs(statevector.data) ** 4)

        metrics_text = (f"QUANTUM METRICS\n"
                        f"======================\n\n"
                        f"Entanglement Entropy: {entanglement:.4f}\n"
                        f"State Fidelity: {fidelity:.4f}\n"
                        f"Quantum Coherence: {coherence:.4f}\n"
                        f"Total Probability: {np.sum(amplitudes):.4f}\n\n"
                        f"Circuit Depth: {result['circuit'].depth()}\n"
                        f"Gate Count: {len(result['circuit'].data)}\n")
        ax5.text(0.1, 0.9, metrics_text, transform=ax5.transAxes,
                 fontsize=11, verticalalignment='top', fontfamily='monospace',
                 bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

        plt.tight_layout()
        plt.show()

        return {'entanglement': entanglement,
                'fidelity': fidelity,
                'coherence': coherence}


In [3]:
class QuantumCircuitOptimizer(nn.Module):
    def __init__(self, input_size=10, hidden_size=64, output_size=6):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()  # Outputs scaled between -pi and pi
        )

    def forward(self, x):
        return self.network(x) * np.pi

def create_variational_circuit(params, n_qubits=3, device=None):
    if device is None:
        device = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(device)
    def circuit(parameters):
        for i in range(n_qubits):
            qml.RY(parameters[i], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        for i in range(n_qubits):
            qml.RZ(parameters[i + n_qubits], wires=i)
        return qml.state()

    return circuit(params)

def ai_optimize_circuit(target_state, simulator, iterations=50):
    print("Starting AI optimization...")
    optimizer_net = QuantumCircuitOptimizer()
    optimizer = optim.Adam(optimizer_net.parameters(), lr=0.01)

    n_qubits = simulator.n_qubits
    if target_state == 'superposition':
        target = np.ones(2 ** n_qubits) / np.sqrt(2 ** n_qubits)
    elif target_state == 'bell':
        target = np.zeros(2 ** n_qubits)
        target[0] = 1 / np.sqrt(2)
        target[-1] = 1 / np.sqrt(2)
    else:
        target = np.zeros(2 ** n_qubits)
        target[0] = 1

    losses = []
    best_params = None
    best_fidelity = 0

    for i in range(iterations):
        circuit_features = torch.randn(1, 10)
        optimized_params = optimizer_net(circuit_features).squeeze()
        state = create_variational_circuit(optimized_params.detach().numpy(), n_qubits, simulator.pennylane_device)
        fidelity = np.abs(np.vdot(state, target)) ** 2
        loss = 1 - fidelity

        optimizer.zero_grad()
        loss_tensor = torch.tensor(loss, requires_grad=True)
        loss_tensor.backward()
        optimizer.step()

        losses.append(loss.item())
        if fidelity > best_fidelity:
            best_fidelity = fidelity
            best_params = optimized_params.detach().numpy()

        if (i + 1) % 20 == 0:
            print(f"Iteration {i + 1}: Fidelity = {fidelity:.4f}, Loss = {loss:.4f}")

    print(f"Optimization complete! Best fidelity: {best_fidelity:.4f}")

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(losses)
    plt.title('Optimization Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss (1 - Fidelity)')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    fidelities = [1 - loss for loss in losses]
    plt.plot(fidelities)
    plt.title('Target State Fidelity')
    plt.xlabel('Iteration')
    plt.ylabel('Fidelity')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return best_params, best_fidelity


In [4]:
def interactive_circuit_builder():
    gates_config = []
    
    gate_type = Dropdown(
        options=['H', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'CNOT'],
        value='H',
        description='Gate:'
    )
    
    qubit_selector = IntSlider(
        value=0,
        min=0,
        max=simulator.n_qubits - 1,
        description='Qubit:'
    )
    
    angle_slider = FloatSlider(
        value=np.pi / 2,      # Default value π/2
        min=0.0,             # Minimum angle 0 radians
        max=2 * np.pi,       # Maximum angle 2π radians (~6.283)
        step=0.01,           # Step for precision
        description='Angle:'
    )
    
    target_qubit = IntSlider(
        value=1,
        min=0,
        max=simulator.n_qubits - 1,
        description='Target:'
    )
    
    add_gate_btn = Button(description='Add Gate', button_style='success')
    clear_btn = Button(description='Clear Circuit', button_style='danger')
    run_btn = Button(description='Run Circuit', button_style='primary')
    optimize_btn = Button(description='AI Optimize', button_style='warning')
    output = widgets.Output()
    
    def add_gate(b):
        with output:
            clear_output(wait=True)
            # Validate control/target qubit for CNOT
            if gate_type.value == 'CNOT' and qubit_selector.value == target_qubit.value:
                print("Control and target qubits cannot be the same for CNOT gate.")
                return
            
            gate_info = {'type': gate_type.value, 'qubit': qubit_selector.value}
            if gate_type.value in ['RX', 'RY', 'RZ']:
                gate_info['angle'] = angle_slider.value
            elif gate_type.value == 'CNOT':
                gate_info['target'] = target_qubit.value
            
            gates_config.append(gate_info)
            print(f"Added {gate_type.value} gate to qubit {qubit_selector.value}")
            print(f"Current gates: {len(gates_config)}")
    
    def clear_circuit(b):
        with output:
            clear_output()
            gates_config.clear()
            print("Circuit cleared")
    
    def run_circuit(b):
        with output:
            clear_output()
            if not gates_config:
                print("No gates in circuit. Add some gates first!")
                return
            
            try:
                print("Running quantum circuit...")
                circuit = simulator.create_circuit(gates_config)
                result = simulator.execute_circuit(circuit)
                simulator.visualize_quantum_state(result)
                print("Circuit execution complete!")
            except Exception as e:
                print(f"Error running circuit: {e}")
    
    def ai_optimize(b):
        with output:
            clear_output()
            print("AI optimization starting...")
            best_params, fidelity = ai_optimize_circuit('superposition', simulator, iterations=50)
            gates_config.clear()
            for i in range(simulator.n_qubits):
                gates_config.append({'type': 'RY', 'qubit': i, 'angle': best_params[i]})
            for i in range(simulator.n_qubits - 1):
                gates_config.append({'type': 'CNOT', 'qubit': i, 'target': i + 1})
            for i in range(simulator.n_qubits):
                gates_config.append({'type': 'RZ', 'qubit': i, 'angle': best_params[i + simulator.n_qubits]})
            print(f"AI optimization complete! Fidelity: {fidelity:.4f}")
            print("Circuit updated with optimized parameters")
    
    add_gate_btn.on_click(add_gate)
    clear_btn.on_click(clear_circuit)
    run_btn.on_click(run_circuit)
    optimize_btn.on_click(ai_optimize)
    
    controls = VBox([
        HBox([gate_type, qubit_selector]),
        HBox([angle_slider, target_qubit]),
        HBox([add_gate_btn, clear_btn]),
        HBox([run_btn, optimize_btn]),
        output
    ])
    
    display(controls)

# Initialize simulator
simulator = QuantumAISimulator(n_qubits=3)

# Launch interactive builder - run this in a separate cell to start UI
interactive_circuit_builder()


Simulator initialized with 3 qubits
